# Programming with Python

## Lecture 08: Class metaprogramming

### Armen Gabrielyan

#### Yerevan State University / ASDS

#### 12 Apr, 2025

## Class metaprogramming

This section is heavily influenced by the following:

*References:*

- Fluent Python, Luciano Ramalho

### Classes are objects

In [146]:
class A:
    pass


class B(A):
    pass


class C(B):
    pass

In [147]:
B.__bases__

(__main__.A,)

In [148]:
B.__subclasses__()

[__main__.C]

In [149]:
B.mro()

[__main__.B, __main__.A, object]

### `type()`

The `type()` function in Python is used to determine the type of an object or dynamically create new classes.

In [150]:
type(42)

int

In [151]:
for t in int, str, list, set, tuple:
    print(type(t))

<class 'type'>
<class 'type'>
<class 'type'>
<class 'type'>
<class 'type'>


In [152]:
class A:
    pass

type(A)

type

In [153]:
type(ValueError)

type

In [154]:
type(type)

type

`type` is a **metaclass**, meaning it is a class that builds other classes.

```
type(name, bases, attributes)
```

- `name`: Name of the class.
- `bases`: Tuple of base classes.
- `attributes`: Dictionary of attributes and methods.

In [155]:
A = type("A", (object,), {"a": 42, "greet": lambda self: "Hello from A!"})
B = type("B", (A,), {"b": 24, "greet": lambda self: "Hello from B!"})

In [158]:
from dis import dis

In [156]:
a = A()
a.a, a.greet()

(42, 'Hello from A!')

In [160]:
dis(A)

Disassembly of greet:
  1           0 RESUME                   0
              2 RETURN_CONST             1 ('Hello from A!')



In [163]:
class A:
#     def __init__(self):
#         self.a = 42
        
    def greet(self):
        return "Hello from A!"

In [164]:
dis(A)

Disassembly of greet:
  5           0 RESUME                   0

  6           2 RETURN_CONST             1 ('Hello from A!')



In [157]:
b = B()
b.b, b.greet()

(24, 'Hello from B!')

### Class factory function

In the following example we define a `record_factory` function that acts like `@dataclass`

In [172]:
from typing import Union, Any
from collections.abc import Iterable, Iterator


FieldNames = str | Iterable[str]

def parse_identifiers(names: FieldNames) -> tuple[str, ...]:
    if isinstance(names, str):
        names = names.replace(',', ' ').split()
    if not all(s.isidentifier() for s in names):
        raise ValueError('names must all be valid identifiers')
    return tuple(names)


def record_factory(cls_name: str, field_names: FieldNames) -> type[tuple]:
    slots = parse_identifiers(field_names)

    def __init__(self, *args, **kwargs) -> None:
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)

    def __iter__(self) -> Iterator[Any]:
        for name in self.__slots__:
            yield getattr(self, name)

    def __repr__(self):
        values = ", ".join(f"{name}={value!r}"
            for name, value in zip(self.__slots__, self))
        cls_name = self.__class__.__name__
        return f"{cls_name}({values})"

    cls_attrs = dict(
        __slots__=slots,
        __init__=__init__,
        __iter__=__iter__,
        __repr__=__repr__,
    )

    return type(cls_name, (object,), cls_attrs)

In [171]:
Person = record_factory("Person", "name age")

('name', 'age')


In [168]:
p = Person(name="John Doe", age=42)
p

Person(name='John Doe', age=42)

In [169]:
name, age = p

print(name)
print(age)

John Doe
42


### `__init_subclass__()` method

The `__init_subclass__()` method is a special class method that is automatically called whenever a subclass is created. It allows a base class to customize the behavior of its subclasses.

In [174]:
from collections.abc import Callable
from typing import Any, NoReturn, get_type_hints


class Field:
    def __init__(self, name: str, constructor: Callable) -> None:
        if not callable(constructor) or constructor is type(None):
            raise TypeError(f'{name!r} type hint must be callable')
        self.name = name
        self.constructor = constructor

    def __set__(self, instance: Any, value: Any) -> None:
        if value is ...:
            value = self.constructor()
        else:
            try:
                value = self.constructor(value)
            except (TypeError, ValueError) as e:
                type_name = self.constructor.__name__
                msg = f'{value!r} is not compatible with {self.name}:{type_name}'
                raise TypeError(msg) from e
        instance.__dict__[self.name] = value

In [175]:
class Checked:
    @classmethod
    def _fields(cls) -> dict[str, type]:
        return get_type_hints(cls)

    def __init_subclass__(subclass) -> None:
        super().__init_subclass__()
        for name, constructor in subclass._fields().items():
            setattr(subclass, name, Field(name, constructor))

    def __init__(self, **kwargs: Any) -> None:
        for name in self._fields():
            value = kwargs.pop(name, ...)
            setattr(self, name, value)
        if kwargs:
            self.__flag_unknown_attrs(*kwargs)

    def __setattr__(self, name: str, value: Any) -> None:
        if name in self._fields():
            cls = self.__class__
            descriptor = getattr(cls, name)
            descriptor.__set__(self, value)
        else:
            self.__flag_unknown_attrs(name)

    def __flag_unknown_attrs(self, *names: str) -> NoReturn:
        plural = 's' if len(names) > 1 else ''
        extra = ', '.join(f'{name!r}' for name in names)
        cls_name = repr(self.__class__.__name__)
        raise AttributeError(f'{cls_name} object has no attribute{plural} {extra}')

    def _asdict(self) -> dict[str, Any]:
        return {
            name: getattr(self, name)
            for name, attr in self.__class__.__dict__.items()
            if isinstance(attr, Field)
        }

    def __repr__(self) -> str:
        kwargs = ', '.join(
            f'{key}={value!r}' for key, value in self._asdict().items()
        )
        return f'{self.__class__.__name__}({kwargs})'

In [176]:
class Person(Checked):
    name: str
    age: int
    salary: float

In [177]:
person = Person(name="John Doe", age=42, salary=199_999.99)
person

Person(name='John Doe', age=42, salary=199999.99)

In [178]:
person.name, person.age

('John Doe', 42)

In [179]:
person.age = "text"

TypeError: 'text' is not compatible with age:int

In [180]:
person = Person()
person

Person(name='', age=0, salary=0.0)

In [181]:
person = Person(first_name="John Doe", age=42, salary=199_999.99)

AttributeError: 'Person' object has no attribute 'first_name'

The `__init_subclass__()` method is called after the class is created. Adding `__slots__` to an existing class has no effect, meaning we cannot use the `__init_subclass__()` method for that purpose.

### Class decorator

A **class decorator** is a decorator that modifies or enhances a class instead of a function. It is similar to a function decorator but applies changes to a class rather than a function.

The main benefit of a class decorator over the simpler `__init_subclass__` is to avoid interfering with other class features, such as inheritance and metaclasses. This is actually why data classes are implemented as a class decorator.

In [ ]:
def _fields(cls: type) -> dict[str, type]:
    return get_type_hints(cls)

def __init__(self: Any, **kwargs: Any) -> None:
    for name in self._fields():
        value = kwargs.pop(name, ...)
        setattr(self, name, value)
    if kwargs:
        self.__flag_unknown_attrs(*kwargs)

def __setattr__(self: Any, name: str, value: Any) -> None:
    if name in self._fields():
        cls = self.__class__
        descriptor = getattr(cls, name)
        descriptor.__set__(self, value)
    else:
        self.__flag_unknown_attrs(name)

def __flag_unknown_attrs(self: Any, *names: str) -> NoReturn:
    plural = 's' if len(names) > 1 else ''
    extra = ', '.join(f'{name!r}' for name in names)
    cls_name = repr(self.__class__.__name__)
    raise AttributeError(f'{cls_name} has no attribute{plural} {extra}')

def _asdict(self: Any) -> dict[str, Any]:
    return {
        name: getattr(self, name)
        for name, attr in self.__class__.__dict__.items()
        if isinstance(attr, Field)
    }

def __repr__(self: Any) -> str:
    kwargs = ', '.join(
        f'{key}={value!r}' for key, value in self._asdict().items()
    )
    return f'{self.__class__.__name__}({kwargs})'

In [ ]:
def checked(cls: type) -> type:
    for name, constructor in _fields(cls).items():
        setattr(cls, name, Field(name, constructor))

    cls._fields = classmethod(_fields)

    instance_methods = (
        __init__,
        __repr__,
        __setattr__,
        _asdict,
        __flag_unknown_attrs,
    )
    for method in instance_methods:
        setattr(cls, method.__name__, method)

    return cls

In [ ]:
@checked
class Person:
    name: str
    age: int
    salary: float

In [ ]:
person = Person(name="John Doe", age=42, salary=199_999.99)
person

In [ ]:
person.name, person.age

### Metaclasses

A **metaclass** in Python is a class of a class. It defines how a class behaves, similar to how a class defines how an instance behaves. In simple terms, a metaclass controls the creation and behavior of classes.

In [ ]:
class A:
    pass

In [ ]:
for t in int, str, list, set, tuple, A, ValueError, type:
    print(t.__class__)

Classes are instances of `type` by default, but they all are subclasses of `object`.

![Type and object relationships](resources/type-object-relationships.png)

### `__new__` method

Metaclasses customize a class through `__new__` method. The `__new__` method in a metaclass is responsible for creating the class before it gets initialized. It allows customization of class creation, modifying attributes, methods, or even rejecting class definitions.

The` __new__` method in a metaclass receives:

- `meta_cls`: The metaclass itself.
- `cls_name`: The name of the class being created.
- `bases`: A tuple of base classes.
- `cls_dict`: A dictionary containing the class attributes and methods.

It must return a new class object using `super().__new__()`.

#### Basic example

In [ ]:
class MyMeta(type):
    def __new__(cls, name, bases, class_dict, **kwargs):
        print(f"Creating class: {name=}")
        print(f"{cls=}")
        print(f"{bases=}")
        print(f"{class_dict=}")
        print(f"{kwargs=}")
        return super().__new__(cls, name, bases, class_dict)

In [ ]:
class MyClass(metaclass=MyMeta, class_param_1=42, class_param_2=True):
    pass

#### Adding an attribute 

In [ ]:
class AutoAttrMeta(type):
    def __new__(cls, name, bases, class_dict):
        class_dict["auto_attr"] = "I was added by metaclass!"

        return super().__new__(cls, name, bases, class_dict)

In [ ]:
class MyClass(metaclass=AutoAttrMeta):
    pass

In [ ]:
MyClass.auto_attr

#### Class name validation

In [ ]:
class NoUnderscoreNamesMeta(type):
    def __new__(cls, name, bases, class_dict):
        if "_" in name:
            raise TypeError("Class names cannot contain underscores!")
        return super().__new__(cls, name, bases, class_dict)

In [ ]:
class ValidClass(metaclass=NoUnderscoreNamesMeta):
    pass

In [ ]:
class Invalid_Class(metaclass=NoUnderscoreNamesMeta):
    pass

### `Checked` with a metaclass

If we want to add `__slots__` to our `Checked` classes (maybe to reduce memory usage and gain performance), we can use a metaclass.

In [ ]:
from collections.abc import Callable
from typing import Any, NoReturn, get_type_hints

In [ ]:
class Field:
    def __init__(self, name: str, constructor: Callable) -> None:
        if not callable(constructor) or constructor is type(None):
            raise TypeError(f'{name!r} type hint must be callable')
        self.name = name
        # Each descriptor instance is a class attribute, and now we need separate per-instance storage attributes.
        self.storage_name = '_' + name
        self.constructor = constructor

    def __get__(self, instance, owner=None):
        # We return either the descriptor instance or the value stored in the attribute named storage_name
        if instance is None:
            return self
        return getattr(instance, self.storage_name)

    def __set__(self, instance: Any, value: Any) -> None:
        if value is ...:
            value = self.constructor()
        else:
            try:
                value = self.constructor(value)
            except (TypeError, ValueError) as e:
                type_name = self.constructor.__name__
                msg = f'{value!r} is not compatible with {self.name}:{type_name}'
                raise TypeError(msg) from e
        setattr(instance, self.storage_name, value) 

In [ ]:
class CheckedMeta(type):
    def __new__(meta_cls, cls_name, bases, cls_dict):
        if '__slots__' not in cls_dict:
            slots = []
            # class does not exist yet, that's why we cannot use typing.get_type_hints
            type_hints = cls_dict.get('__annotations__', {})
            for name, constructor in type_hints.items():
                field = Field(name, constructor)
                cls_dict[name] = field
                slots.append(field.storage_name)

            cls_dict['__slots__'] = slots

        return super().__new__(meta_cls, cls_name, bases, cls_dict) 

In [ ]:
class Checked(metaclass=CheckedMeta):
    __slots__ = ()  # skip CheckedMeta.__new__ processing

    @classmethod
    def _fields(cls) -> dict[str, type]:
        return get_type_hints(cls)

    def __init__(self, **kwargs: Any) -> None:
        for name in self._fields():
            value = kwargs.pop(name, ...)
            setattr(self, name, value)
        if kwargs:
            self.__flag_unknown_attrs(*kwargs)

    def __flag_unknown_attrs(self, *names: str) -> NoReturn:
        plural = 's' if len(names) > 1 else ''
        extra = ', '.join(f'{name!r}' for name in names)
        cls_name = repr(self.__class__.__name__)
        raise AttributeError(f'{cls_name} object has no attribute{plural} {extra}')

    def _asdict(self) -> dict[str, Any]:
        return {
            name: getattr(self, name)
            for name, attr in self.__class__.__dict__.items()
            if isinstance(attr, Field)
        }

    def __repr__(self) -> str:
        kwargs = ', '.join(
            f'{key}={value!r}' for key, value in self._asdict().items()
        )
        return f'{self.__class__.__name__}({kwargs})'

In the `Checked` class:

1. Added an empty `__slots__` to signal to `CheckedMeta.__new__` that this class doesn’t require special processing.
2. Removed `__init_subclass__`. Its job is now done by `CheckedMeta.__new__`.
3. Removed `__setattr__`. It became redundant because adding `__slots__` to the user-defined class prevents setting undeclared attributes.

In [ ]:
class Person(Checked):    
    name: str
    age: int
    salary: float

In [ ]:
person = Person(name="John Doe", age=42, salary=199_999.99)
person

In [ ]:
person.name

In [ ]:
person.name = "Jane Dane"

In [ ]:
person.year = "text"

In [ ]:
person.__dict__

In [ ]:
Person.__dict__

### `__prepare` method

The __prepare__ method is a special class method that is used in the metaclass mechanism. The purpose of `__prepare__` is to return the initial namespace (a dictionary-like object) in which the class body will be executed. The `__prepare__` method should be implemented as a `classmethod`. The namespace returned by `__prepare__` is passed in to `__new__`, but when the final class object is created the namespace is copied into a new `dict`.

In [ ]:
class Meta(type):
    @classmethod
    def __prepare__(meta_cls, cls_name, bases, **kwargs):
        print(f"Preparing namespace for {cls_name}")
        print(f"{meta_cls=}")
        print(f"{bases=}")
        print(f"{meta_cls=}")
        return {}  # You can return any mapping type

    def __new__(meta_cls, cls_name, bases, cls_dict, **kwargs):
        print(f"Creating class {cls_name}")
        return super().__new__(meta_cls, cls_name, bases, cls_dict)  

In [ ]:
class MyClass(metaclass=Meta):
    attr1 = 10
    attr2 = 20

MyClass.attr1

#### Example with `OrderedDict`

In [ ]:
from collections import OrderedDict

class Meta(type):
    @classmethod
    def __prepare__(meta_cls, cls_name, bases):
        return OrderedDict()  # Ensures attributes maintain order

    def __new__(meta_cls, cls_name, bases, cls_dict):
        print("Attributes in order:", list(cls_dict.keys()))
        return super().__new__(meta_cls, cls_name, bases, dict(cls_dict))

class MyClass(metaclass=Meta):
    a = 1
    b = 2
    c = 3

### Metaclasses in the standard library

- `abc.ABCMeta`
- `enum.EnumMeta`
- `collections.NamedTupleMeta`

We usually don't need to use these metaclasses in our code. In general, it is better to make metaclasses as implementation details, so the users of your code can easily make sense of the program. Furthermore, it is good practice to consider if a metaclass is needed at all in your use cases.

In [ ]:
from abc import ABCMeta

from collections.abc import Iterable

In [ ]:
ABCMeta.__class__

In [ ]:
Iterable.__class__

In [ ]:
issubclass(ABCMeta, type)

In [ ]:
issubclass(Iterable, type)

Every class is an instance of `type`, directly or indirectly, but metaclasses are also subclasses of `type`.